### Comment Writing Engine

#### an application to help teachers write better comments in less time



In [6]:
import copy,csv

class Table:
    def __init__(self,header=None,path=None,delimiter=None,data=None):
        """
        Constructs the table object
        """
        self.data = []
        self.header = []
        
        if path and delimiter:
            with open(path) as f:
                if not header:
                    first_line = next(f).strip().split(delimiter)
                    for h in first_line:
                        self.header.append(h.strip())
                else:
                    self.header = header
                for line in f:
                    line = line.strip().split(delimiter)
                    self.data.append({})
                    for index in range(len(line)):
                        self.data[-1][self.header[index]] = line[index]

        if path and not delimiter:
            with open(path) as f:
                reader = csv.DictReader(f)

                for line in reader:
                    self.data.append(line)

                self.header = list(self.data[0].keys())
        
        if header:
            self.header = header
        if data:
            self.data = data
    
    def select(self,columns:list=None):
        """
        Selects given rows as a new table object
        """
        if not columns:
            columns = self.header
        
        data = [{column: row[column] for column in columns} for row in self.data]
        
        return Table(data=data,header=columns)
    
    def typecast(self,columns,newtype):
        """
        Typecasts entire column
        """

        if type(columns) == str:
            columns = [columns]
        
        for index,row in enumerate(self.data):
            for header,cell in row.items():
                for column in columns:
                    if header == column:
                        self.data[index][header] = newtype(cell)
           
        return self
    
    def get_data(self):
        """
        Returns table data as a list of dictionary
        """
        return self.data
    
    def as_lists(self):
        """
        Returns data as list of lists
        """
        returned = []
        
        for row in self.data:
            returned_row = []
            for header,cell in row.items():
                returned_row.append(cell)
            returned.append(returned_row)
        
        return returned
    
    def where(self,condition):
        """
        Filters data given a condition, returns a table with that data
        """
        filtered = []
        
        for index,row in enumerate(self.data):
            if condition(row):
                filtered.append(row)
                
        return Table(data=filtered,header=self.header)
    
    def each(self,expression,dictionary=False):
        """
        Applies a function to each row of a table
        """
        if dictionary:
            returned = {}
            
            for row in self.data:
                returned.update(expression(row))
            
            return returned
            
        returned = []
        
        for row in self.data:
            returned.append(expression(row))
            
        return returned
    
    def add_column(self,data:list,header):
        """
        Takes a list and adds it as a new column
        """
        new_header = self.header
        
        new_header.append(header)
        
        new_data = self.data
        
        for index,row in enumerate(new_data):
            self.data[index][header] = data[index]
            
        return Table(data=new_data,header=new_header)
        
    def get_column(self,column):
        """
        Returns a column as a list
        """
        
        return [row[column] for row in self.data]
    
    def order_by(self,column=None,key=None,reverse=False):
        """
        Sorts table data as new table
        """
        data = copy.deepcopy(self.data)
        
        if column:
            data.sort(key=lambda row: row[column],reverse=reverse)
        elif key:
            data.sort(key=key,reverse=reverse)
        
        return Table(data=data,header=self.header)
    
    def get_row(self,index):
        """
        Returns row at given index
        """
        return self.data[index]
    
    def add_row(self,data):
        """
        Add row to table
        """
        
        self.data.append({header: data[index] for index,header in enumerate(self.header)})
        
        return self
    
    def count(self):
        """
        Returns length of table
        """
        return len(self.data)
    
    def write_to_file(self,path,doublecheck=False):
        """
        Write table to file
        """
        with open(path,'w') as f:
            f.write(",".join(self.header)+'\n')
            self.typecast(self.header,str)
            for row in self.data:
                f.write(','.join(list(row.values()))+'\n')
        if doublecheck:
            with open(path) as f:
                print(f.read())


In [36]:
#############################################################################################################################
# Link to spreadsheet: https://docs.google.com/spreadsheets/d/1XIJnjEZQu0TOO8PnKgoIDXPLMRGeFGlmLt81VXOthQc/edit?usp=sharing #
#############################################################################################################################

students = Table(path='student_info.csv')

def strengths_and_weaknesses(student) -> str: #casey
    """
    Creates a bullet point about a student's strengths and weaknesses given data about that student
    """
    
    comment = ""
    
    if not student["Contributes?"]:
        comment += f"Strength - {student['Strength LO']}\n\n"
    else:
        comment += "Strengths:\n"
        comment += f"\t- {student['Strength LO']}\n"
        
        if student["Contributes?"]:
            comment += f"\t- You are engaged in all assigned tasks. You always contribute in class and your intrinsic motivation is going to serve you well in life\n" 
        
        
        comment += "\n\n"
    
    
    if student["Homework?"] and student["Tardies"] <= 4:
        comment += f"Area of growth - {student['Growth LO']}"
    else:
        comment += "Areas of growth:\n"
        comment += f"\t- {student['Growth LO']}\n"
        
        if not student["Homework?"]:
            comment += f"\t- Furthermore, please make sure you are turning in your homework on time. It will help you understand the class work. \n"
        
        if student["Tardies"] > 4:
            comment += f"\t- On an additional note, I would like you to work on being on time to class. It is imperitive to dive right in immediately as the bell starts."
    
    comment += "\n\n"
    
    return comment


def introductory_line(student) -> str:
    """
    Creates the introductary line about a student using name & grade
    """
    
    statuses = { #Freddy's dictionary
        "A":"an exceptional",
        "A-":"an excellent",
        "B+":"a great",
        "B":"a good",
        "B-":"a good",
        "C+":"a developing",
        "C":"a developing",
        "C-":"a growing",
        "D+": "a challenged",
        "D": "a challenged",
        "D-": "a struggling",
        "F": "a struggling"
    }
    
    comment = f"{student['First Name']}, you are {statuses[student['Grade']]} student. "

    return comment


def improvement(student): #freddy
    """
    creates comments depending on the improvement of the student and their grade
    """
    
    comment = ""
    
    if student["Improvement"]:
        # If improved
        if student["Grade"] in ['A',"A-"]:
            comment += "You improved this semester, and it shows in your final grade. I am very proud of your positive growth this semester and it's clear that your study habits have improved since the start of the year. "
        elif student["Grade"] in ['B+','B','B-']:
            comment += "You have improved a lot this semester and your study habits have undoubtedly improved. I am very proud of the work you've done and I would make sure your work ethic does not falter. "
       
        
        # Challenging note comment
        if float(student["Final Exam %"]) < 85:
            comment += "Unfortunately, a semester of positive growth ended on a challenging note. Hopefully you have some insight as to how your preparation methods were different leading up to this most recent assessment. That aside, you should still be tremendously proud of how you’ve pushed yourself this semester. You are capable of the very highest levels of proficiency. "  
    
    return comment    


def office_hours(student): #freddy's code
    """
    Creates comment depending on the students use of office hours and their grade
    """
    
    comment = ""
    
    if student["Office Hours"]:
        if student["Grade"] in ["A","A-"]:
            comment += "You regularly visit me during office hours, which is reflected in your grades. Office hours are a great resource to not only receive help but strengthen your skills and take them to the next level, beyond the class coursework, which I think you are ready for."
        elif student["Grade"] in ["B-","B","B+"]:
            comment += "You visit office hours regularly which has significantly helped you. Utilizing office hours are a great opportunity to strengthen your class skills. Continue to make consistent use of office hours and peer support to stay on track with the content and skills."
        else:
            comment += "Despite visiting office hours regularly, you may not be fully taking my feedback and I would encourage you to spend time out of class to reflect on what we discuss during this time. Office hours are a great way to stay on track with content. I would recommend meeting with me and reflecting on new strategies we can try."
    else:
        if student["Grade"] in ["A","A-"]:
            comment += f"You independently succeed and have a diligent work ethic in class. As with every student, despite their grade, I would recommend meeting with me during office hours, even if to just take your understanding of the coursework to the next level through aditional, advanced problems. I am very proud of you {student['First Name']} and im excited to see where your {student['Class']} skills take you in the future."
        elif student["Grade"] in ["B-","B","B+"]:
            comment += "A suggestion I would have for you is to meet with me during open block to get valuable feedback. Office hours is a great way for you to keep strengthen your class skills and stay on track with the content. You are a very strong student and with office hours and that little bit of additional support I think you would flourish."
        else:
            comment += "Please visit me during office hours. Office hours is a great way for you to keep strengthen your class skills and stay on track with the content. I would like to meet with you to discuss strategies on how to improve and what strategies we can take to improve your studying skills as it appears that you're not gaining a firm understanding  of the concepts. Come and discuss with me what  your plan of action will be going forward and I look forward to seeing you."
    
    comment += " "
    
    return comment   


def behavior(student):
    """
    Creates a comment if the student's behavior is low enough
    """
    
    comment = ""
    
    if student["Behavior"] <= 2: #casey
        comment += "I would like you to focus more on the learning and less on the and distracting in class. Not only does it not help others but you will struggle to learn as well, despite how you feel about the material. A goal for you is to work on active listening and self-control in the classroom. The frequent redirection and blaming of others when reminded rather than adjusting behavior is harmful to everyone."
    
    return comment

def f_comment(student):
    """
    Creates a comment if the student's grade is an f
    """    
    
    if student["Grade"] == "F": #casey
        comment = "Unfortunately, I strongly suggest you drop this class. Your grades are quite low and I think the level of this classwork is too difficult."
        return comment
    
    return ""


def generate_comment(student):    
    """
    Take in a csv file of student info, creates files for each student with an auto-generated comment depending on variables chosen through spreadsheet
    """    
    
    student = {key:(value.strip() if type(value) == str else value) for key,value in student.items() }
    
    print(student)
    
    print("-"*10 + student["First Name"] + " " + student["Last Name"] + "-"*10 + "\n\n") #casey yo
    
    comment = ""
    
    # Strengths and Weaknesses bullet points #casey
    
    comment += strengths_and_weaknesses(student)
    
    # Introductory line
    
    comment += introductory_line(student)
    
    # Improvement
    
    comment += improvement(student)
    
    # Office hours
    
    comment += office_hours(student)
    
    # behavior
    
    comment += behavior(student)
    
    #f
    
    comment += f_comment(student)
    
    print(comment,'\n')
        
    with open(f"{student['First Name']}{student['Last Name']}.txt",'w') as f: #freddy
        f.write(comment)
    
    


students.typecast(['Office Hours','Improvement','Homework?'],int)
students.typecast(['Office Hours','Improvement','Homework?'],bool)
students.typecast(['Behavior','Tardies'],int)
students.each(generate_comment)

{'First Name': 'Carson', 'Last Name': 'M', 'Class': 'Calc BC', 'Grade': 'F', 'Office Hours': True, 'Improvement': False, 'Final Exam %': '100', 'Behavior': 5, 'Strength LO': 'Being tardy', 'Growth LO': 'I can calculate the area under a curve using antiderivatives', 'Contributes?': '1', 'Homework?': True, 'Tardies': 1}
----------Carson M----------


Strengths:
	- Being tardy
	- You are engaged in all assigned tasks. You always contribute in class and your intrinsic motivation is going to serve you well in life


Area of growth - I can calculate the area under a curve using antiderivatives

Carson, you are a struggling student. Despite visiting office hours regularly, you may not be fully taking my feedback and I would encourage you to spend time out of class to reflect on what we discuss during this time. Office hours are a great way to stay on track with content. I would recommend meeting with me and reflecting on new strategies we can try. Unfortunately, I strongly suggest you drop th

[None, None, None, None]